# **ASL RECOGNITION PROJECT**

In [ ]:
import tensorflow as tf


In [ ]:
!pip install -q kaggle

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle datasets download -d grassknoted/asl-alphabet

100% 1.02G/1.03G [00:06<00:00, 166MB/s]
100% 1.03G/1.03G [00:06<00:00, 162MB/s]


In [ ]:
!unzip asl-alphabet.zip

In [ ]:
# Specifying the training and test directories

TRAINING_DIR = './asl_alphabet_train/asl_alphabet_train/'
TEST_DIR = './asl_alphabet_test/asl_alphabet_test/'

In [ ]:
# Preparing ImageDataGenerator object for training the model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SIZE = 200
BATCH_SIZE = 64

data_generator = ImageDataGenerator(
    samplewise_center=True, 
    samplewise_std_normalization=True,
    brightness_range=[0.8, 1.0],
    zoom_range=[1.0, 1.2],
    validation_split=0.1
)

train_generator = data_generator.flow_from_directory(TRAINING_DIR, 
                                                     target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                                     shuffle=True, 
                                                     seed=13,
                                                     class_mode='categorical',
                                                     batch_size=BATCH_SIZE,
                                                     subset="training")

validation_generator = data_generator.flow_from_directory(TRAINING_DIR,
                                                          target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                                          shuffle=True,
                                                          seed=13,
                                                          class_mode='categorical',
                                                          batch_size=BATCH_SIZE,
                                                          subset="validation")

Found 78300 images belonging to 29 classes.
Found 8700 images belonging to 29 classes.


In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /content/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

In [ ]:
# Loading inception v3 network for transfer learning
from tensorflow.keras import layers
from tensorflow.keras import Model

from tensorflow.keras.applications.inception_v3 import InceptionV3

WEIGHTS_FILE = './inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

inception_v3_model = InceptionV3(
    input_shape = (IMAGE_SIZE, IMAGE_SIZE, 3), 
    include_top = False, 
    weights = 'imagenet'
)
    
# Checking model summary to pick a layer (if required)
inception_v3_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 200, 200, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_376 (Conv2D)            (None, 99, 99, 32)   864         ['input_5[0][0]']                
                                                                                                  
 batch_normalization_376 (Batch  (None, 99, 99, 32)  96          ['conv2d_376[0][0]']             
 Normalization)                                                                                   
                                                                                       

In [ ]:
inception_output = inception_v3_model.output

**Adding our own set of fully connected layers at the end of Inception v3 network:**

In [ ]:
from tensorflow.keras.optimizers import RMSprop, Adam, SGD

x = layers.GlobalAveragePooling2D()(inception_output)
x = layers.Dense(1024, activation='relu')(x)                 
x = layers.Dense(29, activation='softmax')(x)           
model = Model(inception_v3_model.input, x) 

model.compile(
    optimizer=SGD(lr=0.0001, momentum=0.9),
    loss='categorical_crossentropy',
    metrics=['acc']
)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


**Looking at the final model:**

In [ ]:
# Watch the new model summary
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 200, 200, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_188 (Conv2D)            (None, 99, 99, 32)   864         ['input_3[0][0]']                
                                                                                                  
 batch_normalization_188 (Batch  (None, 99, 99, 32)  96          ['conv2d_188[0][0]']             
 Normalization)                                                                                   
                                                                                              

**Setting up a callback funtion in order to stop training at a particular threshold:**

In [ ]:
# Creating a callback to stop model training after reaching a threshold accuracy

LOSS_THRESHOLD = 0.2
ACCURACY_THRESHOLD = 0.95

class ModelCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if logs.get('val_loss') <= LOSS_THRESHOLD and logs.get('val_acc') >= ACCURACY_THRESHOLD:
      print("\nReached", ACCURACY_THRESHOLD * 100, "accuracy, Stopping!")
      self.model.stop_training = True

callback = ModelCallback()

**Fitting the model to the training dataset:**

In [ ]:
history = model.fit_generator(
    train_generator,
    validation_data=validation_generator,
    steps_per_epoch=100,
    validation_steps=50,
    epochs=50,
    callbacks=[callback]
)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  import sys


Epoch 1/50
100/100 [==============================] - 150s 1s/step - loss: 3.3880 - acc: 0.0564 - val_loss: 3.3922 - val_acc: 0.0422
Epoch 2/50
100/100 [==============================] - 111s 1s/step - loss: 3.1636 - acc: 0.1550 - val_loss: 3.1546 - val_acc: 0.1291
Epoch 3/50
100/100 [==============================] - 110s 1s/step - loss: 2.9362 - acc: 0.3086 - val_loss: 2.9391 - val_acc: 0.2722
Epoch 4/50
100/100 [==============================] - 110s 1s/step - loss: 2.6600 - acc: 0.4686 - val_loss: 2.7248 - val_acc: 0.3825
Epoch 5/50
100/100 [==============================] - 108s 1s/step - loss: 2.3399 - acc: 0.5892 - val_loss: 2.4819 - val_acc: 0.4841
Epoch 6/50
100/100 [==============================] - 111s 1s/step - loss: 2.0398 - acc: 0.6776 - val_loss: 2.1509 - val_acc: 0.6109
Epoch 7/50
100/100 [==============================] - 111s 1s/step - loss: 1.7245 - acc: 0.7544 - val_loss: 1.8252 - val_acc: 0.6856
Epoch 8/50
100/100 [==============================] - 108s 1s/step - 

**Training Accuracy vs Validation Accuracy and loss:**

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.legend(loc=0)
plt.figure()


plt.show()

In [ ]:
# Saving the model
MODEL_NAME = 'model3.h5'
model.save(MODEL_NAME)

In [ ]:
from google.colab import files
files.download(MODEL_NAME)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>